## 准备数据集

### 1.cosmosqa

In [5]:
prompt = "You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the options need to be answered, no additional explanations are needed.\n" \
         "An example will be given below:\n" \
         "Text: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .\n" \
         "Question: In the future , will this person go to see other bands play ?\n" \
         "Option 0: None of the above choices .\n" \
         "Option 1: This person likes music and likes to see the show , they will see other bands play .\n" \
         "Option 2: This person only likes Good Old War and Person L , no other bands .\n" \
         "Option 3: Other Bands is not on tour and this person can not see them .\n" \
         "Answer: 1\n\n" \
         "Here are the questions you have to answer: \n"
print(prompt)

You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the options need to be answered, no additional explanations are needed.
An example will be given below:
Text: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .
Question: In the future , will this person go to see other bands play ?
Option 0: None of the above choices .
Option 1: This person likes music and likes to see the show , they will see other bands play .
Option 2: This person only likes Good Old War and Person L , no other bands .
Option 3: Other Bands is not on tour and this person can not see them .
Answer: 1

Here are the questions you have to answer: 



In [6]:
# 转换为 ChatML 格式
import os
import shutil
import json
import csv

input_dir = "cosmosqa-master/data"
output_dir = "data_MiniCPM"

for fn in ["train", "valid"]:
    with open(os.path.join(input_dir, f"{fn}.csv"), "r") as f, open(os.path.join(output_dir, f"cosmosqa_{fn}.json"), "w") as fo:
        reader = csv.DictReader(f)
        for line in reader:
            data_out = {
                "messages": [
                    {
                        "role": "user",
                        "content": f"{prompt}" \
                        f"Text: {line['context']}\n" \
                        f"Question: {line['question']}\n" \
                        f"Option 0: {line['answer0']}\n" \
                        f"Option 1: {line['answer1']}\n" \
                        f"Option 2: {line['answer2']}\n" \
                        f"Option 3: {line['answer3']}\n" \
                        f"Please choose an option."
                    },
                    {
                        "role": "assistant",
                        "content": line["label"],
                    },
                ]
            }
            json.dump(data_out, fo, ensure_ascii=False)
            fo.write('\n')
        

### 2.TriviaQA

In [5]:
prompt_tri = "You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.\n" \
             "Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Answer: York\n" \
             "Here is the question you have to answer:\n"
print(prompt_tri)

You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.
Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Answer: York
Here is the question you have to answer:



In [8]:
from tqdm import tqdm
import os
import shutil
import json

# 读取文件内容的函数
def read_file_content(filepath):
    with open(filepath, 'r') as file:
        return file.read()

txt_path_web = "evidence/web"
txt_path_wikipedia = "evidence/wikipedia"

# 转换为 ChatML 格式
for fn in ["train", "dev"]:
    for wn in ["web"]:
        old_json_path = f"qa/{wn}-{fn}.json"
        txt_path = f"evidence/{wn}"
        if fn == "train":
            new_json_path = f"data_MiniCPM/triviaqa_{wn}_{fn}_withDescription.json"
        else:
            new_json_path = f"data_MiniCPM/triviaqa_{wn}_valid_withDescription.json"

        total_lines = sum(1 for line in open(old_json_path, 'r'))
        with open(old_json_path, 'r', encoding='utf-8') as file, open(new_json_path, "w") as fo:
            # 跳过开头的大括号和"Data": [
            file.readline()
            file.readline()
        
            item_count = 0  # 用于计数处理的 item 数量
            item = {}
            # item['Filename'] = []
            # item['Filename_line'] = []  #debug用
            item['Description'] = []
            
            for line in tqdm(file, total=total_lines - 2, desc="Processing"):
                # 解析 key-value
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip().strip('"')
                    value = value.strip().strip(',').strip('"')
                    value = value.replace('\\','')
                    if key == 'Value':
                        if item_count == 0:
                            pass
                        else:
                            # 导出上一个item的信息
                            # for filename in item['Filename']:
                            #     filepath = os.path.join(txt_path, filename)
                            #     try:
                            #         try:
                            #             item['context'] = read_file_content(os.path.join(txt_path_web, filename))
                            #         except FileNotFoundError:
                            #             item['context'] = read_file_content(os.path.join(txt_path_wikipedia, filename))                                
                            #     except FileNotFoundError:
                            #         print(f"File {filename} not found.")
                            #         # print(item['Filename_line'])
                            #         continue  # 跳过当前步骤，开始下一个循环
    
                            descriptions = "\n".join([f"Description {index+1}: {desc}" for index, desc in enumerate(item['Description'])])
                            data_out = {
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{prompt_tri}" \
                                        f"Question: {item['Question']}\n" \
                                        f"Text: {descriptions}"
                                    },
                                    {
                                        "role": "assistant",
                                        "content": item['Answer'],
                                    },
                                ]
                            }
                            json.dump(data_out, fo, ensure_ascii=False)
                            fo.write('\n')
                            
                            item = {}
                            item['Description'] = []
                            # item['Filename'] = []
                            # item['Filename_line'] = []
                        item_count += 1
                        item['Answer'] = value
        
                    # elif key == 'Filename':
                    #     item['Filename'].append(value)
                    #     item['Filename_line'].append(line)
                        
                    elif key == 'Description':
                        item['Description'].append(value)
                        
                    elif key == 'Question':
                        item['Question'] = value
                        
               # if item_count > 35000:
               #      print("Item count exceeded 35000. Stopping...")
               #      break 
                        

Processing: 100%|██████████| 928801/928801 [00:01<00:00, 588451.15it/s]


In [20]:
import json
import os

cosmos_train_path = "/hy-tmp/data_MiniCPM/cosmosqa_train.json"
cosmos_valid_path = "/hy-tmp/data_MiniCPM/cosmosqa_valid.json"
trivia_train_path = "/hy-tmp/data_MiniCPM/triviaqa_web_train_withDescription.json"
trivia_valid_path = "/hy-tmp/data_MiniCPM/triviaqa_web_valid_withDescription.json"
mix_train_path = "/hy-tmp/data_MiniCPM/mix_train.json"
mix_valid_path = "/hy-tmp/data_MiniCPM/mix_valid.json"

with open(cosmos_train_path, 'r') as cosfile, open(trivia_train_path, 'r') as trifile, open(mix_train_path, 'w') as mixfile:
    for item in range(500):
        mixfile.write(cosfile.readline())
        mixfile.write(trifile.readline())
        
with open(cosmos_valid_path, 'r') as cosfile, open(trivia_valid_path, 'r') as trifile, open(mix_valid_path, 'w') as mixfile:
    for item in range(500):
        mixfile.write(cosfile.readline())
        mixfile.write(trifile.readline())
        

### 以下是构造数据集之前做的一些准备工作

In [1]:
# 看一下train文件的前200行

import json

# 定义一个计数器来跟踪已打印的行数
line_count = 0

# 打开 JSON 文件
with open("qa/web-train.json", 'r') as file:
    for line in file:
        print(line.strip())
            
        # 增加计数器
        line_count += 1
        
        # 如果已经打印了 200 行，则退出
        if line_count >= 200:
            break


{
"Data": [
{
"Answer": {
"Aliases": [
"(Harry) Sinclair Lewis",
"Harry Sinclair Lewis",
"Lewis, (Harry) Sinclair",
"Grace Hegger",
"Sinclair Lewis"
],
"MatchedWikiEntityName": "Sinclair Lewis",
"NormalizedAliases": [
"grace hegger",
"lewis harry sinclair",
"harry sinclair lewis",
"sinclair lewis"
],
"NormalizedMatchedWikiEntityName": "sinclair lewis",
"NormalizedValue": "sinclair lewis",
"Type": "WikipediaEntity",
"Value": "Sinclair Lewis"
},
"EntityPages": [],
"Question": "Which American-born Sinclair won the Nobel Prize for Literature in 1930?",
"QuestionId": "tc_1",
"QuestionSource": "http://www.triviacountry.com/",
"SearchResults": [
{
"Description": "The Nobel Prize in Literature 1930 Sinclair ... The Nobel Prize in Literature 1930 was awarded to ... nobelprize.org/nobel_prizes/literature/laureates/1930/>",
"Filename": "46/46_46.txt",
"Rank": 0,
"Title": "The Nobel Prize in Literature 1930",
"Url": "http://www.nobelprize.org/nobel_prizes/literature/laureates/1930/"
},
{
"Descript

In [2]:
# 看一下train文件的前200行

import json

# 定义一个计数器来跟踪已打印的行数
line_count = 0

# 打开 JSON 文件
with open("qa/wikipedia-train.json", 'r') as file:
    for line in file:
        print(line.strip())
            
        # 增加计数器
        line_count += 1
        
        # 如果已经打印了 200 行，则退出
        if line_count >= 200:
            break


{
"Data": [
{
"Answer": {
"Aliases": [
"Park Grove (1895)",
"York UA",
"Yorkish",
"UN/LOCODE:GBYRK",
"York, UK",
"Eoforwic",
"Park Grove School",
"York Ham",
"The weather in York",
"City of York",
"York, England",
"York, Yorkshire",
"York ham",
"County Borough of York",
"YORK",
"Eoferwic",
"Park Grove Primary School",
"York, North Yorkshire",
"Yoisk",
"York",
"York (England)"
],
"MatchedWikiEntityName": "York",
"NormalizedAliases": [
"york yorkshire",
"eoferwic",
"park grove primary school",
"park grove school",
"weather in york",
"park grove 1895",
"eoforwic",
"county borough of york",
"york uk",
"un locode gbyrk",
"city of york",
"york england",
"york ua",
"york ham",
"york",
"yorkish",
"yoisk",
"york north yorkshire"
],
"NormalizedMatchedWikiEntityName": "york",
"NormalizedValue": "york",
"Type": "WikipediaEntity",
"Value": "York"
},
"EntityPages": [
{
"DocSource": "TagMe",
"Filename": "England.txt",
"Title": "England"
},
{
"DocSource": "TagMe",
"Filename": "Judi_Dench.txt",
"Title"

In [11]:
# 看一下dev文件的前200行

import json

# 定义一个计数器来跟踪已打印的行数
line_count = 0

# 打开 JSON 文件
with open("qa/wikipedia-dev.json", 'r') as file:
    for line in file:
        print(line.strip())
            
        # 增加计数器
        line_count += 1
        
        # 如果已经打印了 200 行，则退出
        if line_count >= 200:
            break


{
"Data": [
{
"Answer": {
"Aliases": [
"Sunset Blvd",
"West Sunset Boulevard",
"Sunset Boulevard",
"Sunset Bulevard",
"Sunset Blvd."
],
"MatchedWikiEntityName": "Sunset Boulevard",
"NormalizedAliases": [
"sunset boulevard",
"sunset bulevard",
"west sunset boulevard",
"sunset blvd"
],
"NormalizedMatchedWikiEntityName": "sunset boulevard",
"NormalizedValue": "sunset boulevard",
"Type": "WikipediaEntity",
"Value": "Sunset Boulevard"
},
"EntityPages": [
{
"DocSource": "TagMe",
"Filename": "Andrew_Lloyd_Webber.txt",
"Title": "Andrew Lloyd Webber"
}
],
"Question": "Which Lloyd Webber musical premiered in the US on 10th December 1993?",
"QuestionId": "tc_33",
"QuestionSource": "http://www.triviacountry.com/"
},
{
"Answer": {
"Aliases": [
"Sir Henry Campbell-Bannerman",
"Campbell-Bannerman",
"Campbell Bannerman",
"Sir Henry Campbell Bannerman",
"Henry Campbell Bannerman",
"Henry Campbell-Bannerman"
],
"MatchedWikiEntityName": "Henry Campbell-Bannerman",
"NormalizedAliases": [
"henry campbell b

In [1]:
# import json

# # 文件路径
# file_paths = ['data_MiniCPM/cosmosqa/train.json', 'data_MiniCPM/triviaqa/web_train.json', 'data_MiniCPM/triviaqa/wikipedia_train.json']
# output_file_path = 'data_MiniCPM/merged.json'


# # 初始化输出文件
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     output_file.write('[')  # 写入 JSON 数组的起始符并换行

#     first_item = True
#     for file_index, file_path in enumerate(file_paths):
#         with open(file_path, 'r', encoding='utf-8') as input_file:
#             lines = input_file.readlines()
#             total_lines = len(lines)
#             # 跳过第一行
#             for i, line in enumerate(lines[1:], start=2):
#                 if line:
#                     if file_index == len(file_paths) - 1 and i == total_lines:
#                         # 如果是最后一个文件的最后一行，不添加逗号
#                         output_file.write('\n' + line)
#                     else:
#                         if i == len(lines) - 1:
#                             # 如果是文件的最后一行，添加逗号和换行符
#                             output_file.write(',\n')
#                         else:
#                             # 不是最后一行的情况
#                             output_file.write('\n' + line)


In [2]:
# import json

# # 文件路径
# file_paths = ['data_MiniCPM/cosmosqa/valid.json', 'data_MiniCPM/triviaqa/web_dev.json', 'data_MiniCPM/triviaqa/wikipedia_dev.json']
# output_file_path = 'data_MiniCPM/merged_dev.json'


# # 初始化输出文件
# with open(output_file_path, 'w', encoding='utf-8') as output_file:
#     output_file.write('[')  # 写入 JSON 数组的起始符并换行

#     first_item = True
#     for file_index, file_path in enumerate(file_paths):
#         with open(file_path, 'r', encoding='utf-8') as input_file:
#             lines = input_file.readlines()
#             total_lines = len(lines)
#             # 跳过第一行
#             for i, line in enumerate(lines[1:], start=2):
#                 if line:
#                     if file_index == len(file_paths) - 1 and i == total_lines:
#                         # 如果是最后一个文件的最后一行，不添加逗号
#                         output_file.write('\n' + line)
#                     else:
#                         if i == len(lines) - 1:
#                             # 如果是文件的最后一行，添加逗号和换行符
#                             output_file.write(',\n')
#                         else:
#                             # 不是最后一行的情况
#                             output_file.write('\n' + line)
